In [0]:
%%bash 
git clone https://github.com/vaishleshik/histo_rep

Cloning into 'histo_rep'...


In [0]:
!pip install tensorflow-gpu==2.0.0-beta1

In [1]:
import tensorflow as tf 

print(tf.__version__)

2.0.0-beta1


In [0]:
import pandas as pd
import numpy as np
import random
import os
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
%matplotlib inline

# For fixing the randomness and thereby enabling my team members to reproduce the experiments
np.random.seed(7)

import warnings
warnings.filterwarnings("ignore")

In [0]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Model

In [0]:
input_tensor = tf.keras.layers.Input(shape=(256, 192, 3))

base_model = ResNet50(input_tensor=input_tensor,weights='imagenet',include_top=False)

for layer in base_model.layers:
    layer.trainable=False

x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D(data_format='channels_last')(x)
x = tf.keras.layers.Dense(7, activation='softmax')(x)

updatedModel = Model(base_model.input, x)


In [0]:
from tensorflow.keras import optimizers

updatedModel.compile(loss='sparse_categorical_crossentropy',
              optimizer=optimizers.Adam(lr=1e-3),
              metrics=['accuracy'])

In [12]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255,
        horizontal_flip=True)

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255,
        horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
    directory=r"histo_rep/Train/",
    target_size=(256, 192),
    color_mode="rgb",
    batch_size=32,
    class_mode="sparse",
    shuffle=True,
    seed=42
)

test_generator = test_datagen.flow_from_directory(
    directory=r"histo_rep/Test/",
    target_size=(256, 192),
    color_mode="rgb",
    batch_size=32,
    class_mode="sparse",
    shuffle=True,
    seed=42
)

Found 363 images belonging to 7 classes.
Found 81 images belonging to 7 classes.


In [13]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=test_generator.n//test_generator.batch_size

updatedModel.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=test_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=10)

Epoch 1/10
11/11 [==============================] - 7s 630ms/step - loss: 1.2516 - accuracy: 0.5921 - val_loss: 2.1629 - val_accuracy: 0.1094
Epoch 2/10
11/11 [==============================] - 2s 191ms/step - loss: 0.3785 - accuracy: 0.9456 - val_loss: 2.1055 - val_accuracy: 0.1094
Epoch 3/10
11/11 [==============================] - 2s 213ms/step - loss: 0.1440 - accuracy: 0.9758 - val_loss: 2.1182 - val_accuracy: 0.1250
Epoch 4/10
11/11 [==============================] - 2s 223ms/step - loss: 0.0796 - accuracy: 0.9972 - val_loss: 2.1529 - val_accuracy: 0.1094
Epoch 5/10
11/11 [==============================] - 2s 187ms/step - loss: 0.0726 - accuracy: 0.9935 - val_loss: 2.1748 - val_accuracy: 0.1094
Epoch 6/10
11/11 [==============================] - 2s 190ms/step - loss: 0.0611 - accuracy: 0.9970 - val_loss: 2.1774 - val_accuracy: 0.1250
Epoch 7/10
11/11 [==============================] - 2s 180ms/step - loss: 0.0412 - accuracy: 1.0000 - val_loss: 2.2225 - val_accuracy: 0.1094
Epoch 